In [12]:
import cv2
import numpy as np
import random
import time
from keras.models import load_model
from keras.preprocessing.image import img_to_array

# モデルをロード
model_path = "fine_tuned_model.hdf5"
try:
    model = load_model(model_path)
except Exception as e:
    print(f"モデルの読み込みに失敗しました: {e}")
    exit()

# 感情ラベル
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise","Neutral"]
valid_questions =  ["Angry", "Happy", "Sad"]  # 問題候補

# OpenCVの顔検出器をロード
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Webカメラを起動
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    exit()

# 画面サイズ調整
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cv2.namedWindow("Expression Game", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Expression Game", 1280, 720)

# 最初の問題を出題
current_question = random.choice(valid_questions)
while True:
    # フレームを取得
    ret, frame = cap.read()
    if not ret:
        break

    # グレースケール変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顔を検出
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    user_expression = "Unknown"  # デフォルト値

    for (x, y, w, h) in faces:
        try:
            # 顔領域を抽出
            face = gray[y:y+h, x:x+w]
            face = cv2.resize(face, (64, 64))  # モデルの入力サイズに合わせる
            face = face.astype("float32") / 255.0  # 正規化
            face = img_to_array(face)
            face = np.expand_dims(face, axis=0)

            # 感情を予測
            preds = model.predict(face)


            if preds is not None and preds.shape[0] > 0:
                index = np.argmax(preds[0])  # 最大確率のラベルを取得
                user_expression = emotion_labels[index]
            else:
                user_expression = "Unknown"  # 出力が不正な場合

        except Exception as e:
            print(f"Prediction error: {e}")
            user_expression = "Unknown"

        # 顔の枠を描画
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # 予測された感情を表示
        cv2.putText(frame, user_expression, (x, max(y-10, 30)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # 出題中の表情
    cv2.putText(frame, f"Make this face: {current_question}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)

    # 正解した場合
    if user_expression == current_question:
        
        cv2.putText(frame, "Correct!", (500, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 5)
        cv2.imshow("Expression Game", frame)
        cv2.waitKey(2000)  # 2秒待つ
        # 新しい問題を出す
        current_question = random.choice(valid_questions)
        

    # 画面に表示
    cv2.imshow("Expression Game", frame)

    # `q` を押すと終了
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [1]:
import cv2
import numpy as np
import random
import time
from keras.models import load_model
from keras.preprocessing.image import img_to_array

# モデルをロード
model_path = "fine_tuned_model.hdf5"
try:
    model = load_model(model_path)
except Exception as e:
    print(f"モデルの読み込みに失敗しました: {e}")
    exit()

# 感情ラベル
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
valid_questions = ["Angry", "Happy", "Sad"]  # 出題する感情

threshold = 0.3

# OpenCVの顔検出器をロード
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Webカメラを起動
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    exit()

# 画面サイズ調整
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cv2.namedWindow("Expression Game", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Expression Game", 1280, 720)

# 最初の問題を出題
current_question = random.choice(valid_questions)

while True:
    # フレームを取得
    ret, frame = cap.read()
    if not ret:
        break

    # グレースケール変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顔を検出
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    user_expression = "Unknown"  # デフォルト値

    for (x, y, w, h) in faces:
        try:
            # 顔領域を抽出
            face = gray[y:y+h, x:x+w]
            face = cv2.resize(face, (64, 64))  # モデルの入力サイズに合わせる
            face = face.astype("float32") / 255.0  # 正規化
            face = img_to_array(face)
            face = np.expand_dims(face, axis=0)

            # 感情を予測
            preds = model.predict(face)

            if preds is not None and preds.shape[0] > 0:
                index = np.argmax(preds[0])  # 最大確率のラベルを取得
                confidence = preds[0][index]  # 予測の確信度

                # 閾値を超えた場合のみ判定
                if confidence >= threshold:
                    user_expression = emotion_labels[index]
                else:
                    user_expression = "Unknown"  # 確信度が低いため不明

            else:
                user_expression = "Unknown" 

        except Exception as e:
            print(f"Prediction error: {e}")
            user_expression = "Unknown"

        # 顔の枠を描画
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # 予測された感情を表示
        cv2.putText(frame, f"{user_expression} ({confidence:.2f})", (x, max(y-10, 30)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # 出題中の表情
    cv2.putText(frame, f"Make this face: {current_question}", (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)

    # 正解した場合（閾値を超えていることも確認）
    if user_expression == current_question:
        cv2.putText(frame, "Correct!", (500, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 5)
        cv2.imshow("Expression Game", frame)
        cv2.waitKey(2000)  # 2秒待つ
        # 新しい問題を出す
        current_question = random.choice(valid_questions)

    # 画面に表示
    cv2.imshow("Expression Game", frame)

	# `q` を押すと終了
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# 終了処理
cap.release()
cv2.destroyAllWindows()


Using TensorFlow backend.
c:\Users\mkyan\anaconda3\envs\face_tf14\lib\site-packages\keras\models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


KeyboardInterrupt: 